## Figures

The figures were generated below in R.

In [ ]:
#############################
# Install & load R packages #
#############################

#install.packages("readr")
#install.packages("ggplot2")
#install.packages("plotly")
#install.packages("vegan")
#install.packages("calibrate")

library(readr)
library(ggplot2)
library(plotly)
library(vegan)
library(calibrate)

In [ ]:
#######################################
# Read in and full table of TC scores # 
#######################################

tc_scores <- read.csv('../results/tables/full_table_tc.csv', header=F)
colnames(tc_scores) <- c("tag","type","align_method","tree_method","family","nseqs","tc_score")

In [ ]:
############################################
# Function to calculate cumlative variance #
############################################

cum_var <- function(x) {
        ind_na <- !is.na(x)
        nn <- cumsum(ind_na)
        x[!ind_na] <- 0
        cumsum(x^2) / (nn-1) - (cumsum(x))^2/(nn-1)/nn
}

In [ ]:
#################################################################
# Cum. Avg. and Var. of Non-Regressive ClustalO with mBed trees #
#################################################################

nonreg <- tc_scores[tc_scores$type == 'std_align',]
nonreg <- nonreg[nonreg$nseqs > 1000,]
nonreg_co<- nonreg[nonreg$align_method == 'CLUSTALO',]
nonreg_co_co<- nonreg_co[nonreg_co$tree_method == 'CLUSTALO',]
nonreg_co_co_full<- nonreg_co_co[nonreg_co_co$tag == 'full',]
nonreg_co_co_ref<- nonreg_co_co[nonreg_co_co$tag == 'ref',]
nonreg_co_co_full_sorted <- nonreg_co_co_full[order(-nonreg_co_co_full$nseqs),]
nonreg_co_co_ref_sorted <- nonreg_co_co_ref[order(-nonreg_co_co_ref$nseqs),]
nonreg_co_co_diff <-  nonreg_co_co_full_sorted$tc_score - nonreg_co_co_ref_sorted$tc_score 
nonreg_co_co_cumsum <- cumsum(nonreg_co_co_diff)
nonreg_co_co_cumavg<-nonreg_co_co_cumsum/seq_along(nonreg_co_co_cumsum)
nonreg_co_co_cumsd<-sqrt(cum_var(nonreg_co_co_cumavg))
nonreg_co_co_cumsd[is.na(nonreg_co_co_cumsd)] <- 0

#############################################################
# Cum. Avg. and Var. of Regressive ClustalO with mBed trees #
#############################################################

reg <- tc_scores[tc_scores$type == 'dpa_align',]
reg <- reg[reg$nseqs > 1000,]
reg_co<- reg[reg$align_method == 'CLUSTALO',]
reg_co_co<- reg_co[reg_co$tree_method == 'CLUSTALO',]
reg_co_co_full<- reg_co_co[reg_co_co$tag == 'full',]
reg_co_co_full_sorted <- reg_co_co_full[order(-reg_co_co_full$nseqs),]
reg_co_co_diff <-  reg_co_co_full_sorted$tc_score - nonreg_co_co_ref_sorted$tc_score 
reg_co_co_cumsum <- cumsum(reg_co_co_diff)
reg_co_co_cumavg<- reg_co_co_cumsum/seq_along(reg_co_co_cumsum) 
reg_co_co_cumsd<-sqrt(cum_var(reg_co_co_cumavg))
reg_co_co_cumsd[is.na(reg_co_co_cumsd)] <- 0


############################################################
# Cum. Avg. and Var. of Regressive G-INS-1 with mBed trees #
############################################################

reg <- tc_scores[tc_scores$type == 'dpa_align',]
reg <- reg[reg$nseqs > 1000,]
reg_gin<- reg[reg$align_method == 'MAFFT-GINSI',]
reg_gin_co<- reg_gin[reg_gin$tree_method == 'CLUSTALO',]
reg_gin_co_full<- reg_gin_co[reg_gin_co$tag == 'full',]
nonreg_gin<- nonreg[nonreg$align_method == 'MAFFT-GINSI',]
nonreg_gin_co<- nonreg_gin[nonreg_gin$tree_method == 'CLUSTALO',]
nonreg_gin_co_ref<- nonreg_gin_co[nonreg_gin_co$tag == 'ref',]
nonreg_gin_co_ref_sorted <- nonreg_gin_co_ref[order(-nonreg_gin_co_ref$nseqs),]

reg_gin_co_full_sorted <- reg_gin_co_full[order(-reg_gin_co_full$nseqs),]
reg_gin_co_diff <-  reg_gin_co_full_sorted$tc_score - nonreg_gin_co_ref_sorted$tc_score 
reg_gin_co_cumsum <- cumsum(reg_gin_co_diff)
reg_gin_co_cumavg<- reg_gin_co_cumsum/seq_along(reg_gin_co_cumsum) 
reg_gin_co_cumsd<-sqrt(cum_var(reg_gin_co_cumavg))
reg_gin_co_cumsd[is.na(reg_gin_co_cumsd)] <- 0

In [ ]:
#######################################################
# Figure 2A - Average Differential in Total Column Score #
#######################################################
options(scipen = 999)
Figure_2A <- ggplot(nonreg_co_co_full_sorted,aes(x=nonreg_co_co_full_sorted$nseqs)) +

    geom_ribbon(aes(ymin = nonreg_co_co_cumavg - nonreg_co_co_cumsd, 
                ymax = nonreg_co_co_cumavg + nonreg_co_co_cumsd, 
                fill = "nonreg_co_co_cumavg"), alpha= 0.2) +
    geom_line(aes(y = nonreg_co_co_cumavg), color = "#6586D1") +

    geom_ribbon(aes(ymin = reg_co_co_cumavg - reg_co_co_cumsd, 
                ymax = reg_co_co_cumavg + reg_co_co_cumsd, 
                fill = "reg_co_co_cumavg"), alpha= 0.2) +
    geom_line(aes(y = reg_co_co_cumavg), color = "#419836") +

    geom_ribbon(aes(ymin = reg_gin_co_cumavg - reg_gin_co_cumsd, 
                ymax = reg_gin_co_cumavg + reg_gin_co_cumsd, 
                fill = "reg_gin_co_cumavg"), alpha= 0.2) +
    geom_line(aes(y = reg_gin_co_cumavg), color = "#CF6925") +

    xlab("log(Nseq)") + ylab("Score (TC)") +

    scale_x_log10(name="Number of sequences", breaks=c(1000,2500, 10000, 25000, 100000)) +

    theme(axis.line = element_line(colour = "black"),legend.position = "none") +

    scale_fill_manual("",values=c("#6586D1","#419836","#CF6925"))

ggplotly(Figure_2A)

In [ ]:
###################################################
# Figure 2B - Constrained Correspondence Analysis #
###################################################


CCAfigure=function(tablename="../results/tables/figure2b.csv",figurename="../results/figures/CCA.png",legend=FALSE)

{

	ma=read.table(tablename,sep=",",header=TRUE)
	
	# produce indicator matrix #########################	
	s=list()
	d=0
	h="accuracy"
	for (i in 1:3){
		s[[i]]=unique(ma[,i])
		d=d+length(s[[i]])
		h=c(h,as.character(s[[i]]))	
	}
	M=matrix(0,dim(ma)[1],d+1)
	colnames(M)=h
	for (i in 1:dim(ma)[1]){
		M[i,1]=ma[i,"accuracy"]
		ind=2 #next column in M to be filled
		for (j in 1:3){
			for (k in 1:length(s[[j]])){
				
				if (ma[i,j]==as.character(s[[j]][k])){
					M[i,ind]=1
				} else{
					M[i,ind]=0
				}
				ind=ind+1
			}
		}
		
	}

	# produce CCA of indicator matrix ################### 
	m=M[,-1]
	a=M[,1]
	res=cca(m,a)

	# make the figure ###################################
	reg=which(m[,"Regressive"]==1)
	pro=which(m[,"Regressive"]==0)
	
	#scale the accuracy axis:
	x=(sort(res$CCA$u)[length(a)]-sort(res$CCA$u)[1])/(sort(a)[length(a)]-sort(a)[1])
	pc=seq(from=floor(sort(a)[1]/10)*10,to=ceiling(sort(a)[length(a)]/10)*10,by=10)
	a1=sort(res$CCA$u)[1]
	off1=(sort(a)[1]-pc[1])*x
	a2=sort(res$CCA$u)[length(a)]
	off2=(pc[length(pc)]-sort(a)[length(a)])*x
	sc=seq(from=a1-off1,to=a2+off2,by=10*x)

	png(figurename,width=17,height=14,res=300,unit="cm")
	plot(c(res$CCA$u,res$CCA$v),c(res$CA$u[,1],res$CA$v[,1]),type="n",xaxt="n",xlab="Accuracy (%)",ylab="CA 1",asp=1,xlim=c(sc[1],sc[length(sc)]))
	axis(side=1,at=sc,labels=pc,tick=TRUE,cex.axis=1)
	axis(side=3,at=seq(floor(range(c(res$CCA$u,res$CCA$v))[1]),ceiling(range(c(res$CCA$u,res$CCA$v))[2]),1))
	mtext("CCA", side = 3,line=3)
	points(res$CCA$u[pro],res$CA$u[pro,1],pch=1,col="grey",cex=0.8)
	points(res$CCA$u[reg],res$CA$u[reg,1],pch=19,col="grey",cex=0.8)
	st=1
	co=c("#CF6925","#6586D1","grey30")
	for (j in 1:3){
		en=st+length(s[[j]])-1	
		if (j==3){#avoid label clash
			text(res$CCA$v[st],res$CA$v[st,1],colnames(m)[st],pos=2,col=co[j],cex=0.7)
			arrows(0,0,res$CCA$v[st],res$CA$v[st,1],col=co[j],length=0.07,angle=10)
			st=st+1
		
		}
		textxy(res$CCA$v[st:en],res$CA$v[st:en,1],colnames(m)[st:en],col=co[j],cex=0.7)
		arrows(0,0,res$CCA$v[st:en],res$CA$v[st:en,1],col=co[j],length=0.07,angle=10)
		st=en+1
	}
	if (legend==TRUE){
		legend("bottomleft",pch=c(17,17,17,19,1),c("assembler","tree","aligner","regr. aln","non-regr. aln"),col=c(co,"grey","grey"),cex=0.7,bty="n")
	}
	dev.off()
	
	# calculate % variance explained ######################	
	cca=round(res$CCA$eig/(sum(res$CA$eig)+res$CCA$eig)*1000)/10
	ca=round(res$CA$eig[1]/(sum(res$CA$eig)+res$CCA$eig)*1000)/10
	print(paste("CCA axis:",cca,"%variance",sep=" ",collapse=""))
	print(paste("CA axis:",ca,"%variance",sep=" ",collapse=""))
}

CCAfigure()

![CCA Figure](../results/figures/CCA.png)

In [ ]:
###################################################################
# Figure 3A - CPU & Score Changes in Regressive vs Non-regressive #
###################################################################



table<- read.csv(file="../results/tables/figure3b.csv",sep=',',header=T)

fig_3A<-ggplot(table, aes(x =table$cpu, y =table$score, label=table$method)) + 
  geom_line(aes(line=table$method),linetype = "dotted",size=0.8,color='grey',alpha=0.95) +
  geom_point(aes(shape=table$aggregation,color=table$aggregation),size=5.5) +
  labs(shape='', color='')+
  labs(x ="CPU (ms)", y="Total Column Score Accuracy (%)")+
  scale_x_log10(labels = scales::comma)+
  scale_shape_manual(values=c(15,16))+
  scale_color_manual(values = alpha(c("#6586D1","#CF6925")))+
  theme_bw()+
  theme(legend.position = "top", legend.direction = "horizontal")+
  guides(colour = guide_legend(reverse=T),shape = guide_legend(reverse=T))
  #Uncommend to see the labels
  #geom_text()

fig_3A



In [ ]:
########################################
# Read in and full table of CPU times  # 
########################################

cpuTime <- read.csv('../results/tables/full_table_cpu.csv',  header=F, sep=",", stringsAsFactors=F)
colnames(cpuTime) <- c("tag","type","align_method","tree_method","family","nseqs","cpu_time")

####################################################
# Figure 3B - CPU Time of ClustalO with mBed trees #
####################################################

cpuTime_dpa <- cpuTime[cpuTime$nseqs > 10000 & cpuTime$tag == 'full' & cpuTime$type == 'dpa_align' & cpuTime$align_method == 'CLUSTALO' & cpuTime$tree_method == 'CLUSTALO', ] 
cpuTime_std <- cpuTime[cpuTime$nseqs > 10000 & cpuTime$tag == 'full' & cpuTime$type == 'std_align' & cpuTime$align_method == 'CLUSTALO' & cpuTime$tree_method == 'CLUSTALO', ] 
cpuTime_merge <- merge(cpuTime_dpa, cpuTime_std, by=c("tag","align_method", "tree_method", "family"))

cpuTime_merge$cpu_time.x <- as.numeric(cpuTime_merge$cpu_time.x)
cpuTime_merge$cpu_time.y <- as.numeric(cpuTime_merge$cpu_time.y)

cpuTime_merge


#cpuTime_merge$group <- ifelse(cpuTime_merge$nseqs.y > 10000, "above10000", ifelse(cpuTime_merge$nseqs.y < 1000, "below1000", "1000to10000"))


ggplot(cpuTime_merge, aes(x=cpu_time.y, y=cpu_time.x)) +
  geom_smooth( method="lm", se=F,fullrange=TRUE, color= "grey30" ) +
  geom_point(aes(x=cpu_time.y, y=cpu_time.x),shape=16,color="#CF6925", size =1.5) +
  geom_abline(intercept = 0 ,linetype="dashed") +
  scale_x_continuous(labels = scales::comma,limits = c(0, 3200000))+
  scale_y_continuous(labels = scales::comma,limits = c(0, 3200000)) +
  coord_cartesian(xlim=c(0,3200000), ylim=c(0,3200000)) +
  xlab(expression(atop("CPU time (ms) for ClustalO/mBed/Non-regressive"))) +
  ylab(expression(atop("CPU time (ms) for ClustalO/mBed/Regressive"))) +
  theme(text = element_text(size=15),legend.position = c(0.3, 0.85)) +
  theme_bw()+
  theme( axis.line = element_line(colour = "black"), aspect.ratio=1, legend.position="none")

linearMod<-lm(cpuTime_merge$cpu_time.y ~ cpuTime_merge$cpu_time.x, data=cpuTime_merge)
summary(linearMod)